In [1]:
import salabim as sim

## With Queue

In [5]:
class CustomerGenerator(sim.Component):
    def process(self):
        while True:
            Customer()
            self.hold(sim.Uniform(5, 15).sample())


class Customer(sim.Component):
    def process(self):
        if len(waitingline) >= 5:
            env.number_balked += 1
            env.print_trace("", "", "balked")
            print(env.now(), "balked",self.name())
            self.cancel()
        self.enter(waitingline)
        worktodo.trigger(max=1)
        self.hold(50) # if not served within this time -> renege
        if self in waitingline:
            self.leave(waitingline)
            env.number_reneged += 1
            env.print_trace("", "", "reneged")
        else:
            self.passivate() # waiting for service to be completed
        self.passivate()

class Clerk(sim.Component):
    def process(self):
        while True:
            while len(waitingline) == 0:
                self.standby()
                self.wait((worktodo, True, 1))
            self.customer = waitingline.pop()
            self.hold(30)
            self.customer.activate() # signal the customer that all is done



env = sim.Environment(trace=True)

CustomerGenerator()
env.number_balked = 0
env.number_reneged = 0
clerks = [Clerk() for _ in range(3)]
waitingline = sim.Queue('waitingline')
worktodo = sim.State('worktodo')



env.run(duration=3000) # till can also be used
waitingline.length.print_histogram(30,0,1)
waitingline.length_of_stay.print_histogram(30,0,10)
print(f'number balked: {env.number_balked}')
print(f'number reneged: {env.number_reneged}')

waitingline.print_histograms()
worktodo.print_histograms()

# waitingline.print_info()
# print()
# waitingline.print_statistics()

## With Store

In [ ]:

class CustomerGenerator(sim.Component):
    def process(self):
        while True:
            #Customer().enter(waiting_room)
            customer = Customer()
            self.to_store(waiting_room, customer, fail_at=env.now())
            if self.failed():
                customer.cancel()
                env.number_balked += 1
                print(env.now(), "balked",customer.name())
                env.print_trace("", "", "balked",customer.name())
            self.hold(sim.Uniform(5, 15).sample())


class Customer(sim.Component):
    def process(self):
        self.hold(50)
        if self in waiting_room:
            self.leave(waiting_room)
            env.number_reneged += 1
            env.print_trace("", "", "reneged")


class Clerk(sim.Component):
    def process(self):
        while True:
            customer = self.from_store(waiting_room)
            self.hold(30)
            

env = sim.Environment(trace=True)

env.number_balked = 0
env.number_reneged = 0

CustomerGenerator()
for _ in range(3):
    Clerk()
waiting_room = sim.Store('waiting_room', capacity=5)


env.run(till=5000)
waiting_room.length.print_histogram(30, 0, 1)
waiting_room.length_of_stay.print_histogram(30, 0, 10)
print("number reneged", env.number_reneged)
print("number balked", env.number_balked)


## With Resources

In [ ]:
class CustomerGenerator(sim.Component):
    def process(self):
        while True:
            Customer()
            self.hold(sim.Uniform(5, 15).sample())


class Customer(sim.Component):
    def process(self):
        if len(clerks.requesters()) >= 5:
            env.number_balked += 1
            env.print_trace("", "", "balked")
            self.cancel()
        self.request(clerks, fail_delay=50)
        if self.failed():
            env.number_reneged += 1
            env.print_trace("", "", "reneged")
        else:
            self.hold(30)
            self.release()



env = sim.Environment(trace=True)
CustomerGenerator()
env.number_balked = 0
env.number_reneged = 0
clerks = sim.Resource("clerks", capacity=3)

env.run(till=5000)
clerks.print_statistics()
clerks.print_info()

clerks.requesters().length.print_histogram(30, 0, 1)
print()
clerks.requesters().length_of_stay.print_histogram(30, 0, 10)
print("number reneged", env.number_reneged)
print("number balked", env.number_balked)


